In [8]:
import random
import time
import copy

# Karger's random contractions algorithm for computing minimum cuts
This algorithm takes a Monte Carlo approach to compute the minimum cut of an undirected graph. 

### Vertex class for building graphs

In [1]:
class Vertex:
    """
    Vertex class for graph construction
    """
    
    def __init__(self, node):
        """
        Constructore
        :param node: unique, hashable identifier
        """
        # ID
        self.id = node
        # Dict for connected vertices
        self.adjacent = {}
        
    def add_neighbor(self, neighbor, weight=0):
        """
        Add a weighted link to a vertex, making it adjacent
        :param neighbor: Unique hashable identifier for the vertext to connect to
        :param weight: weight of the link, optional.
        :return: None
        """
        self.adjacent[neighbor] = weight
        return None

    def get_connections(self):
        """
        Get the identifiers for connected vertices
        :return: keys of connected vertices
        """
        return self.adjacent.keys()  

    def get_id(self):
        """
        Get key, or ID or a vertex
        :return:
        """
        return self.id
    
    def get_weight(self, neighbor):
        """
        Get the weight of link to a neighbor
        :param neighbor: Unique identifier for the connected node
        :return: weight of the connecting edge, -1 if not connected
        """
        if neighbor in self.adjacent:
            return self.adjacent[neighbor]
        else:
            print("Vertices not connected!")
            return -1


### Graph class

In [2]:
class Graph:
    
    def __init__(self):
        """
        Constructor. Equipped for Karger's random contractions algorithm
        """
        # Dict of vertices in the graph
        self.verts = {}
        # List for edges
        self.edges = []
        # Number of vretices in the graph
        self.num_verts = 0
        
    def add_vert(self, node):
        """
        Add a vertex to the graph
        :param node: unique, hashable identifier for the vertex to introduce to the graph
        :return: None
        """
        if node not in self.verts:
            self.verts[node] = Vertex(node)
            self.num_verts += 1
        return None
    
    def remove_vert(self, node):
        """
        Remove a vertex from the graph.
        :param node: unique, hashable identifier for the vertex to introduce to the graph
        :return: None
        """
        if node in self.verts:
            self.verts.pop(node)
        return None
    
    def get_vertices(self):
        """
        Get the keys of all vertices in the graph
        :return: vertex keys
        """
        return self.verts.keys()
    
    def get_vert(self, node):
        """
        Get the Vertex object corresponding to unique node ID passed
        :param node: unique, hashable identifier for the vertex to introduce to the graph
        :return: Vertex object, if node corresponds to a Vertex in the graph
        """
        if node in self.verts:
            return self.verts[node]
        else:
            return None
        
    def add_edge(self, start, end, weight = 0):
        """
        Add a weighted edge to the graph.
        :param start: unique, hashable identifier for the starting vertex of the edge
        :param end: unique, hashable identifier for the end vertex of the edge
        :param weight: weight of the edge to add, optional
        :return: None
        """
        
        if start not in self.verts:
            self.add_vert(start)
        
        if end not in self.verts:
            self.add_vert(end)
            
        self.verts[start].add_neighbor(self.verts[end], weight)
        self.verts[end].add_neighbor(self.verts[start], weight)
        
        if {start, end} not in self.edges:
            self.edges.append({start, end})
        return None
        
    
    # Edge contraction method
    def contract_edge(self, start, end):
        """
        Contract an edge in the graph
        :param start: ID for vertex at the start of the edge to contract
        :param end: ID for vertex at the end of the edge to contract
        :return: None
        """
        
        if (start not in self.verts) or (end not in self.verts):
            print("Edge nodes not in graph")
            return None
        
        if {start, end} not in self.edges:
            print("Edge not in graph")
            return None
        
        self.remove_vert(end)
        self.num_verts = self.num_verts - 1
        
        #Iterate over all edges. Those linked to the end node are now linked to the start
        for i in range(len(self.edges)):
            
            if end in self.edges[i]:
                self.edges[i].remove(end)
                self.edges[i].add(start)
            
        #Remove any self loops that may result
        while {start} in self.edges:
            self.edges.remove({start})

### Read in the input array

In [3]:
arr = []

with open("kargerMinCut.txt") as file:
    for l in file.readlines():
        arr.append([int(i) for i in l.split('\t')[:-1]])

nodes = [a.pop(0) for a in arr]
edgs = arr.copy()

In [4]:
# Build the graph
graph_in = Graph()
for n, es in zip(nodes, edgs):
    for e in es:
        graph_in.add_edge(n, e)

In [9]:
#Number of random seeds to run the algo with
num_iter = 10
i = 0

# Establish an initial min cut size
min_cut = float("inf")

begin = time.time()

while i < num_iter:
    
    # Get a copy of the original graph 
    graph = copy.deepcopy(graph_in)
    
    # Iteratively contract edges until only two vertices remain 
    while graph.num_verts > 2:
        s, e = graph.edges[random.randint(0,len(graph.edges)-1)]
        graph.contract_edge(s,e)
    
    # Keep track of the smallest cut size
    if len(graph.edges) < min_cut:
        min_cut = len(graph.edges)
    i += 1

print(min_cut)
print('Kargers algorithm ran in', time.time() - begin, 'seconds. Input size: ', len(arr))

17
Kargers algorithm ran in 3.4542675018310547 seconds. Input size:  200
